In [ ]:
! pip install imutils

In [ ]:
import matplotlib

##### THIS GIVES SOME WARNING #####
# matplotlib.use("Agg") 
##### THIS GIVES SOME WARNING #####

from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K 
# import the necessary packages

from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from imutils import paths
import numpy as np
import argparse
import random
import pickle
import os
import cv2

from imutils import paths

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# initialize the number of epochs to train for, initial learning rate,
# batch size, and image dimensions
EPOCHS = 100
INIT_LR = 1e-3
BS = 32
IMAGE_DIMS = (32, 32, 3)
 
# initialize the data and labels
data = []
labels = []

##### IMPORT DRIVE PATH HERE #####
##### IMPORT DRIVE PATH HERE #####
##### IMPORT DRIVE PATH HERE #####
##### IMPORT DRIVE PATH HERE #####
##### IMPORT DRIVE PATH HERE #####
train_data_dir = "/content/gdrive/My Drive/ASL/asl_alphabet_train"
##### IMPORT DRIVE PATH HERE #####
##### IMPORT DRIVE PATH HERE #####
##### IMPORT DRIVE PATH HERE #####
##### IMPORT DRIVE PATH HERE #####
##### IMPORT DRIVE PATH HERE #####

# grab the image paths and randomly shuffle them
print("[INFO] loading images...")
imagePaths = sorted(list(paths.list_images(train_data_dir)))
random.seed(42)
random.shuffle(imagePaths)
print("[INFO] IMAGES LOADED")

In [ ]:
!mv "/content/gdrive/My Drive/ASL/asl_alphabet_train/" DATASET/

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
with open('/gdrive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /gdrive/foo.txt

In [ ]:
!ls /

In [ ]:
imagePaths[4]

In [ ]:
number_of_images = 0

# loop over the input images
for imagePath in imagePaths:
	# load the image, pre-process it, and store it in the data list
	image = cv2.imread(imagePath)
	image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
	image = img_to_array(image)
	data.append(image)
 
	# extract the class label from the image path and update the
	# labels list
	label = imagePath.split(os.path.sep)[-2]
	labels.append(label)

	if (number_of_images%1000 == 0):
		print(number_of_images)
	number_of_images = number_of_images + 1

In [ ]:
data = np.array(data, dtype="float") / 255.0

labels = np.array(labels)

print("[INFO] data matrix: {:.2f}MB".format(
	data.nbytes / (1024 * 1000.0)))

print("BACH GAYEE BABU!!!")

In [ ]:
# binarize the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
 
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.2, random_state=42)

# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")


In [ ]:
# initialize the model
print("[INFO] compiling model...")


model = Sequential()
inputShape = (32, 32, 3)
chanDim = -1
 
# if we are using "channels first", update the input shape
# and channels dimension
if K.image_data_format() == "channels_first":
	inputShape = (depth, height, width)
	chanDim = 1

model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
        

model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
        
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
 

model.add(Dense(29))
model.add(Activation("softmax"))
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

print("[INFO] compiling model...DONE!!!")

In [ ]:
# train the network
print("[INFO] training network...")
H = model.fit_generator(
	aug.flow(trainX, trainY, batch_size=BS),
  validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS, verbose=1)

In [ ]:
model.save('test_model.h5')

In [ ]:
image = cv2.imread("/home/sumanu/Desktop/asl/asl-alphabet/asl_alphabet_train/del/del1.jpg")
output = image.copy()
 
# pre-process the image for classification
image = cv2.resize(image, (32, 32))
image = image.astype("float") / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)
print(model.predict(image))